<a href="https://colab.research.google.com/github/seopp/Tobigs19/blob/main/Week4/tobigs19_KNN_%E1%84%80%E1%85%AA%E1%84%8C%E1%85%A6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 19기 KNN 정규세션 과제

## KNN 구현해보기
### 1. Preprocssing / EDA
지금까지 배운 내용을 토대로 해당 데이터에 대해 자유롭게 전처리와 EDA를 진행해주세요.
### 2. KNN 구현 & 파라미터 튜닝
수업 내용 및 실습 자료를 참고하여 KNN을 구현하고 파라미터 튜닝을 하며 결과를 비교해주세요.
### 3. Evaluation
결과에 대한 평가를 진행하고, 나름의 해석을 달아주세요.

**데이터:** [blackfriday | Kaggle](https://www.kaggle.com/llopesolivei/blackfriday)

---

## 0. 데이터 불러오기

In [28]:
import pandas as pd
df = pd.read_csv("blackfriday.csv", index_col = 0)
df.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1001088,P00046042,F,0-17,10,A,3,0,5,17.0,NaN,2010
1,1004493,P00347742,F,0-17,10,A,1,0,7,NaN,NaN,4483
2,1005302,P00048942,F,0-17,10,A,1,0,1,4.0,NaN,7696
3,1001348,P00145242,F,0-17,10,A,3,0,2,4.0,NaN,16429
4,1001348,P00106742,F,0-17,10,A,3,0,3,5.0,NaN,5780


In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4998 entries, 0 to 4997
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   User_ID                     4998 non-null   int64  
 1   Product_ID                  4998 non-null   object 
 2   Gender                      4998 non-null   object 
 3   Age                         4998 non-null   object 
 4   Occupation                  4998 non-null   int64  
 5   City_Category               4998 non-null   object 
 6   Stay_In_Current_City_Years  4998 non-null   object 
 7   Marital_Status              4998 non-null   int64  
 8   Product_Category_1          4998 non-null   int64  
 9   Product_Category_2          3465 non-null   float64
 10  Product_Category_3          1544 non-null   float64
 11  Purchase                    4998 non-null   int64  
dtypes: float64(2), int64(5), object(5)
memory usage: 507.6+ KB


In [30]:
df.tail()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
4993,1000837,P00147942,M,55+,7,C,1,1,1,16.0,NaN,11701
4994,1000293,P00358242,M,55+,1,C,1,1,8,NaN,NaN,2045
4995,1001908,P00173342,M,55+,13,C,3,1,1,6.0,14.0,4505
4996,1005517,P00336742,M,55+,0,C,2,1,12,NaN,NaN,1413
4997,1003015,P00141842,M,55+,6,C,0,1,5,14.0,NaN,7182


## 1.PreProcessing / EDA

In [31]:
# ID랑 결측치 많은 칼럼인 Product_Category_3 드랍
df = df.drop(columns=["User_ID", "Product_ID","Product_Category_3"])

Encoding

In [32]:
df["Gender"] = df["Gender"].map(lambda x: 1 if x=="M" else 0)

In [33]:
features = ['Age','City_Category','Stay_In_Current_City_Years']

In [34]:
from sklearn.preprocessing import LabelEncoder

In [35]:
for o_col in features:
    encoder = LabelEncoder()
    encoder.fit(df[o_col])
    df[o_col] = encoder.transform(df[o_col])

결측치 처리


In [36]:
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=3)
imputed= imputer.fit_transform(df)
df = pd.DataFrame(imputed, columns=df.columns)

Split

In [37]:
from sklearn.model_selection import train_test_split
y = df["Purchase"]
X = df.drop(columns=["Purchase"])

In [38]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.3)

Scaling

In [39]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## 2. KNN 구현 & 파라미터 튜닝

KNN

In [40]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

In [41]:
knn = KNeighborsRegressor()
knn.fit(X_train, y_train)
pred = knn.predict(X_test)

In [42]:
mse = mean_squared_error(y_test, pred)
print(f"MSE: {mse}")

MSE: 22031938.752613332


GridSearchCV

In [43]:
#그리드서치를 위한 파라미터 설정
params = {
    "n_neighbors": [i for i in range(1, 20, 2)],
    "p": [1, 2], #1은 맨하탄, 2는 유클리드
    "weights": ['uniform']
}

In [44]:
grid_cv = GridSearchCV(knn, param_grid=params, cv=3)
grid_cv.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=KNeighborsRegressor(),
             param_grid={'n_neighbors': [1, 3, 5, 7, 9, 11, 13, 15, 17, 19],
                         'p': [1, 2], 'weights': ['uniform']})

In [45]:
grid_cv.best_params_
knn_grid = KNeighborsRegressor(**grid_cv.best_params_)
knn_grid.fit(X_train, y_train)

KNeighborsRegressor(n_neighbors=11, p=1)

## 3. Evaluation

In [46]:
preds = knn_grid.predict(X_test)
mse = mean_squared_error(y_test, preds)
print(f"MSE: {mse}")

MSE: 19428796.428435262


그리드 서치를 통해 MSE를 줄일 수 있었음